In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from amftrack.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests
from mpl_toolkits.mplot3d import Axes3D

In [2]:
window=800
infos = pickle.load(open(f'{path_code}/MscThesis/Results/straight_bait_{window}.pick', "rb"))

In [3]:
infos.to_csv(f'{path_code}/MscThesis/Results/straight_bait_{window}.csv')

In [23]:
(infos['vx']**2).astype(float)

0       11025.0
1       24649.0
2       10816.0
3       20736.0
4       30976.0
         ...   
8254     1600.0
8255    53361.0
8256    42849.0
8257    98596.0
8258     3249.0
Name: vx, Length: 8259, dtype: float64

In [24]:
infos['abs_angle_to_P']=np.abs(infos['angle_to_P'])
infos['inv_dens']=1/infos['density']
infos['side_cross']=infos.apply (lambda row: comments[row['plate']] if row['plate'] in comments.keys() else 'None', axis=1)
blur = 10
infos['spvgd']=(infos['vx']*infos[f'grad_density_x{blur}']+infos['vy']*infos[f'grad_density_y{blur}'])/np.sqrt((infos['vx']**2+infos['vy']**2).astype(float))

In [11]:
corrected = infos.loc[(infos["straightness"] <= 1)& (infos["speed"] >=25)& (infos["speed"] <400)&(infos["straightness"] > 0.95)&(infos["density"]>0)]

In [7]:
corrected['abs_angle_to_P']=np.abs(corrected['angle_to_P'])
corrected['inv_dens']=1/corrected['density']
corrected['side_cross']=corrected.apply (lambda row: comments[row['plate']] if row['plate'] in comments.keys() else 'None', axis=1)
# corrected['residual']=corrected['speed']-f(corrected['inv_dens'])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [6]:
corrected

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,dist_N,angle_to_P,angle_to_N,t,hyph,abs_angle_to_P,side_cross,side_P_bait
1,351,"(35, 0, 15)",25,18.091350,0.063647,0.925362,284.245706,71.061426,0.956046,34520.041525,22307.329306,-115.775736,132.013276,0.0,18,115.775736,None,None
3,351,"(35, 0, 15)",25,-10.236040,-0.040914,0.850000,250.186905,62.546726,0.994030,35106.987977,22538.780688,-122.946939,127.129363,12.0,18,122.946939,None,None
4,351,"(35, 0, 15)",25,10.731002,0.033386,0.844203,321.420439,80.355110,0.988017,35226.293042,22724.596184,-135.382220,115.455469,16.0,18,135.382220,None,None
5,351,"(35, 0, 15)",25,2.423771,0.009316,0.905797,260.161868,65.040467,0.956446,35449.164207,22837.868690,-134.832478,117.066914,20.0,18,134.832478,None,None
6,351,"(35, 0, 15)",25,-2.428157,-0.008189,0.996377,296.517106,74.129276,0.987698,35555.161080,22949.549433,-148.352615,104.458145,24.0,18,148.352615,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8252,424,"(13, 116, 131)",baits,7.281669,0.012852,0.903623,566.586163,141.646541,0.968746,47451.023860,16077.494264,37.023983,-65.400035,56.0,10105,37.023983,left,right
8253,424,"(13, 116, 131)",baits,-20.905807,-0.030679,2.035507,681.432714,170.358178,0.984990,46972.393814,15827.563481,39.025778,-65.624935,60.0,10105,39.025778,left,right
8255,424,"(13, 116, 131)",baits,24.176796,0.050994,1.960870,474.114541,118.528635,0.992319,45906.669249,14455.405413,-130.594397,116.497577,60.0,11031,130.594397,left,right
8256,424,"(13, 116, 131)",baits,16.548582,0.029624,2.907971,558.616681,139.654170,0.981037,46240.456212,14676.851572,-150.235475,99.643155,64.0,11031,150.235475,left,right


In [20]:
get_Pside(395).values[0]

'right'

In [5]:
def get_Pside(plate_number):
    return(plate_info.loc[plate_info['Plate #'] == plate_number]['P-bait'])
path_code = "/home/cbisot/pycode/"
plate_info = pd.read_excel(path_code + 'MscThesis/plate_info/SummaryAnalizedPlates.xlsx',engine='openpyxl',header=3,)
corrected['side_P_bait']=corrected.apply (lambda row: get_Pside(row['plate']).values[0] if row['plate'] in comments.keys() else 'None', axis=1)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [72]:
baits = corrected.loc[corrected ['treatment']=='baits']

In [57]:
baits

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,dist_N,angle_to_P,angle_to_N,t,hyph,abs_angle_to_P,inv_dens,side_cross,residual
3913,395,"(10, 0, 10)",baits,6.554181,0.028963,1.000725,226.294882,56.573721,0.990876,15350.350868,44853.955251,-51.626442,-170.251998,16.0,99,51.626442,0.999276,ignore,NaN
3914,395,"(10, 0, 10)",baits,-11.309932,-0.030246,0.857971,373.937647,93.484412,0.986906,15196.256183,45088.839997,-49.022529,-167.205634,20.0,99,49.022529,1.165541,ignore,NaN
3915,395,"(10, 0, 10)",baits,4.466079,0.008599,0.728261,519.382846,129.845711,0.996801,14969.600490,45535.581956,-50.207648,-167.382359,24.0,99,50.207648,1.373134,ignore,NaN
3916,395,"(10, 0, 10)",baits,-4.467159,-0.007155,0.587681,624.316099,156.079025,0.991340,14598.383522,45908.993600,-54.918511,-170.866374,28.0,99,54.918511,1.701603,ignore,NaN
3917,395,"(10, 0, 10)",baits,12.260655,0.016484,0.657246,743.776576,185.172591,0.996342,14278.909935,46657.081149,-58.661901,-172.566551,32.0,99,58.661901,1.521499,ignore,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8252,424,"(13, 116, 131)",baits,7.281669,0.012852,0.903623,566.586163,141.646541,0.968746,47451.023860,16077.494264,37.023983,-65.400035,56.0,10105,37.023983,1.106656,left,-104.521580
8253,424,"(13, 116, 131)",baits,-20.905807,-0.030679,2.035507,681.432714,170.358178,0.984990,46972.393814,15827.563481,39.025778,-65.624935,60.0,10105,39.025778,0.491278,left,-37.823611
8255,424,"(13, 116, 131)",baits,24.176796,0.050994,1.960870,474.114541,118.528635,0.992319,45906.669249,14455.405413,-130.594397,116.497577,60.0,11031,130.594397,0.509978,left,-93.484518
8256,424,"(13, 116, 131)",baits,16.548582,0.029624,2.907971,558.616681,139.654170,0.981037,46240.456212,14676.851572,-150.235475,99.643155,64.0,11031,150.235475,0.343882,left,-57.152447


In [35]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(column = ['angle_to_P'],by="plate",figsize =(9,8),ax =ax,patch_artist=True, showfliers=False, notch=True,showmeans = True)
colors = 9*['lightblue']+ 16*['pink']
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [5]:
fig = plt.figure()
ax = plt.axes(projection='3d')
X,Y,Z = baits['dist_P'], baits['angle_to_P'], baits['speed']
ax.plot_trisurf(X, Y, Z,
                cmap='viridis', edgecolor='none');
# ax.scatter3D(baits['dist_P'], baits['angle_to_P'], baits['speed'], c=baits['speed'], cmap='Greens');

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
compressa = 100
compressb = 1
count = np.zeros((70000//compressa,360//compressb,))
color_map = np.zeros((70000//compressa,360//compressb,))
# color_map[:] = np.nan

In [20]:
baits.values

array([[395, (10, 0, 10), 'baits', ..., 158.13292123944194, 16.0, 99],
       [395, (10, 0, 10), 'baits', ..., 161.43978079510393, 20.0, 99],
       [395, (10, 0, 10), 'baits', ..., 161.44764937842342, 24.0, 99],
       ...,
       [424, (13, 116, 131), 'baits', ..., -143.43800607976135, 60.0,
        11031],
       [424, (13, 116, 131), 'baits', ..., -166.24713695505574, 64.0,
        11031],
       [424, (13, 116, 131), 'baits', ..., 146.41584795291794, 64.0,
        12330]], dtype=object)

In [69]:
baits_copy = baits.copy()
for index,row in baits_copy.iterrows():
    new_row = row.copy()
    if row['angle_to_P']>0:
        new_row['angle_to_P']-=360
    else:
        new_row['angle_to_P']+=360
    if row['angle_to_N']>0:
        new_row['angle_to_N']-=360
    else:
        new_row['angle_to_N']+=360
    baits = baits.append(new_row)


In [73]:
for index,row in baits.iterrows():
    x = int(row['dist_P'])//compressa
    y = (int(row['angle_to_P'])+180)//compressb
#     print(row['dist_P'])
    color_map[x,y]+=row['residual']
    count[x,y]+=1
# color_map

In [74]:
compressed = (color_map/count)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [75]:
compressed[compressed==0]=np.nan

In [76]:
filtered=scipy.ndimage.generic_filter(compressed,np.nanmean,size=(100,50))

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/scipy/ndimage/filters.py:1528: RuntimeWarning: Mean of empty slice
  cval, origins, extra_arguments, extra_keywords)


In [77]:
fig = plt.figure()
ax = plt.axes()
im=ax.imshow(filtered,cmap='Greens')
labels = ['-180', '-90', '0','90','180']
ax.set_xticks([0,90,180,270,360])
ax.set_xticklabels(labels)
ticks = [0,100,200,300,400,500,600,700]
labels = [str(tick/10) for tick in ticks]
ax.set_yticks(ticks)
ax.set_yticklabels(labels)
ax.set_xlabel('angle to P (°)')
ax.set_ylabel('Distance from P (mm)')
cbar = fig.colorbar(im)
cbar.set_label('speed ($\mu m.h^{-1}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
filtered[pos[0],pos[1]]

102.93925554679475

In [40]:
x,y,z=[],[],[]
for pos in np.argwhere(1- np.isnan(filtered)):
    x.append(pos[0])
    y.append(pos[1])
    z.append(filtered[pos[0],pos[1]])


In [19]:
x[:10],y[:10],z[:10]

TypeError: 'int' object is not subscriptable

In [41]:
plt.close('all')
fig = plt.figure()
ax = plt.axes(projection='3d')
# X,Y,Z=baits['dist_P'], baits['angle_to_P'], baits['speed']
# ax.plot_trisurf(x, y, z,
#                 cmap='viridis', edgecolor='none');
ax.scatter3D(x, y, z, c=z, cmap='Greens');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [105]:
no_baits = corrected .loc[corrected ['treatment']=='25']

In [34]:
baits

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,dist_P,dist_N,angle_to_P,angle_to_N,t,hyph,abs_angle_to_P,side_cross,side_P_bait
3913,395,"(10, 0, 10)",baits,6.554181,0.028963,1.000725,226.294882,56.573721,0.990876,15350.350868,44853.955251,-51.626442,-170.251998,16.0,99,51.626442,ignore,right
3914,395,"(10, 0, 10)",baits,-11.309932,-0.030246,0.857971,373.937647,93.484412,0.986906,15196.256183,45088.839997,-49.022529,-167.205634,20.0,99,49.022529,ignore,right
3915,395,"(10, 0, 10)",baits,4.466079,0.008599,0.728261,519.382846,129.845711,0.996801,14969.600490,45535.581956,-50.207648,-167.382359,24.0,99,50.207648,ignore,right
3916,395,"(10, 0, 10)",baits,-4.467159,-0.007155,0.587681,624.316099,156.079025,0.991340,14598.383522,45908.993600,-54.918511,-170.866374,28.0,99,54.918511,ignore,right
3917,395,"(10, 0, 10)",baits,12.260655,0.016484,0.657246,743.776576,185.172591,0.996342,14278.909935,46657.081149,-58.661901,-172.566551,32.0,99,58.661901,ignore,right
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8252,424,"(13, 116, 131)",baits,7.281669,0.012852,0.903623,566.586163,141.646541,0.968746,47451.023860,16077.494264,37.023983,-65.400035,56.0,10105,37.023983,left,right
8253,424,"(13, 116, 131)",baits,-20.905807,-0.030679,2.035507,681.432714,170.358178,0.984990,46972.393814,15827.563481,39.025778,-65.624935,60.0,10105,39.025778,left,right
8255,424,"(13, 116, 131)",baits,24.176796,0.050994,1.960870,474.114541,118.528635,0.992319,45906.669249,14455.405413,-130.594397,116.497577,60.0,11031,130.594397,left,right
8256,424,"(13, 116, 131)",baits,16.548582,0.029624,2.907971,558.616681,139.654170,0.981037,46240.456212,14676.851572,-150.235475,99.643155,64.0,11031,150.235475,left,right


In [22]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(baits['dist_P'],baits['speed'])
ax.set_ylim(0,400)
# ax.set_xlim(26000,32000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0, 400)

In [11]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(np.abs(baits['angle_to_P']),baits['speed'])
ax.set_ylim(0,400)
# ax.set_xlim(26000,32000)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0, 400)

In [72]:
lambd = 10000
baits['gradient']=np.cos(baits['angle_to_P'])*baits['dist_P']*np.exp(-(baits['dist_P']/lambd)**2)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [74]:
baits_sort = baits.sort_values('gradient')

In [57]:
baits_sort.rolling(100,min_periods=1).mean()

,angle,curvature,density,growth,speed,straightness,dist_P,dist_N,angle_to_P,angle_to_N,t,gradient
3980,2.826822,0.002160,0.928261,1308.674252,325.811017,0.993061,7208.961437,7208.961437,53.364389,53.364389,32.000000,-4283.293841
6065,-5.824231,-0.008594,1.478623,1028.399366,256.812347,0.985860,7166.236079,7166.236079,50.211813,50.211813,36.000000,-4281.459074
3981,-2.833782,-0.004949,1.312077,1133.397578,283.157731,0.989899,7084.308754,7084.308754,53.343712,53.343712,36.005556,-4278.369633
4478,0.465150,0.000302,1.527899,1011.392250,252.704315,0.991251,7235.756110,7235.756110,10.930175,10.930175,49.833333,-4270.299744
5871,-0.319830,-0.000526,1.906667,989.245393,247.196350,0.990204,7067.064352,7067.064352,25.714969,25.714969,42.263333,-4265.453995
...,...,...,...,...,...,...,...,...,...,...,...,...
4048,2.294570,0.003823,2.812616,914.868609,214.812279,0.987378,7808.249983,7808.249983,-9.470454,-9.470454,49.439667,3532.262652
4406,2.391351,0.003898,2.779841,916.415030,215.187144,0.987354,7832.749936,7832.749936,-9.600941,-9.600941,49.832833,3544.372686
6092,2.295229,0.003737,2.804986,912.169403,214.132514,0.987153,7833.266665,7833.266665,-7.533915,-7.533915,49.274167,3556.576449
5980,2.763804,0.004267,2.806043,916.542541,215.236552,0.986880,7841.021869,7841.021869,-5.013371,-5.013371,48.715333,3569.040237


In [39]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
plate = 405
selected = baits.loc[baits['side_cross']!='side_P_bait']
# selected = baits
baits_sort = selected.sort_values("angle_to_P")
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('angle to P (°)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
ax.set_xlim(-190,190)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(selected["angle_to_P"],selected['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["angle_to_P"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["angle_to_P"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["angle_to_P"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
plate = 405
selected = baits.loc[baits['side_cross']!='side_P_bait']
# selected = baits
baits_sort = selected.sort_values("angle_to_N")
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('angle to N (°)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
ax.set_xlim(-190,190)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(selected["angle_to_N"],selected['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["angle_to_N"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["angle_to_N"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["angle_to_N"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [111]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
abcisse = 'inv_dens'
ordinate = 'speed'
tab = no_baits
baits_sort = tab.sort_values(abcisse)
N=100
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('inverse density')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
# ax.set_xlim(400,1500)
# ax.set_ylim(-150,150)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
for plate in set(tab['plate'].values):
    select = tab.loc[tab['plate']==plate]
    ax.scatter(select[abcisse],select[ordinate],label=plate,alpha=0.3)
ax.plot(moving_av[abcisse],moving_av[ordinate],color='green',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std')
# plt.legend(loc=2,ncol = 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [112]:
f(0.75)

array(226.87444036)

In [49]:
moving_av['speed']
from scipy.interpolate import interp1d
x = moving_av["inv_dens"]
y = moving_av["speed"]
f = interp1d(x, y,copy=True,fill_value=(150,240),bounds_error=False)

In [61]:
moving_av[abcisse]

6276         NaN
6679         NaN
6285         NaN
7086         NaN
6277         NaN
          ...   
5932    6.224638
4037    6.249906
4990    6.277502
6112    6.308872
6052    6.341184
Name: density, Length: 2956, dtype: float64

In [66]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
baits_sort = baits.sort_values("inv_dens")
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('inverse density')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,400)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(baits["inv_dens"],baits['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["inv_dens"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["inv_dens"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["inv_dens"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
angle = "angle_to_P"
baits_sort = baits.sort_values(angle)
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel(angle +'  (°)')
ax.set_ylabel('inverse density')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,2)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(baits[angle],baits['inv_dens'],color='yellow',label='observed speeds')
ax.plot(moving_av[angle],moving_av['inv_dens'],color='green',label = 'moving average')
ax.plot(moving_av[angle],(moving_av['inv_dens']+moving_std['inv_dens']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av[angle],(moving_av['inv_dens']-moving_std['inv_dens']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
baits_sort = baits.sort_values("angle_to_P")
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('angle to P (°)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,400)
ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(baits["angle_to_P"],baits['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["angle_to_P"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["angle_to_P"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["angle_to_P"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1377: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:237: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:239: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


In [46]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
lambd = 10000
baits['gradient']=np.cos(baits['angle_to_P'])*baits['dist_P']*np.exp(-(baits['dist_P']/lambd)**2)
baits_sort = baits.sort_values("angle_to_N")
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('angle to N (°)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,400)
ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(baits["angle_to_N"],baits['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["angle_to_N"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["angle_to_N"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["angle_to_N"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
lambd = 100000
baits['gradient']=np.cos(baits['angle_to_P'])*baits['dist_P']*np.exp(-(baits['dist_P']/lambd)**2)
baits_sort = baits.sort_values('dist_P')
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('gradient')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(baits['dist_P'],baits['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av['dist_P'],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av['dist_P'],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av['dist_P'],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
baits_sort = no_baits.sort_values("angle_to_P")
N = 100
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('gradient')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(no_baits["angle_to_P"],no_baits['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["angle_to_P"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["angle_to_P"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["angle_to_P"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
baits_sort = no_baits.sort_values("angle_to_N")
N = 100
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('angle to left bait (°)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(no_baits["angle_to_N"],no_baits['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["angle_to_N"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["angle_to_N"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["angle_to_N"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
lambd = 100000
no_baits['gradient']=np.cos(no_baits['angle_to_P'])*no_baits['dist_P']*np.exp(-(no_baits['dist_P']/lambd)**2)
baits_sort = no_baits.sort_values("abs_angle_to_P")
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('gradient')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(no_baits["abs_angle_to_P"],no_baits['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["abs_angle_to_P"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["abs_angle_to_P"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["abs_angle_to_P"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
lambd = 100000
no_baits['gradient']=np.cos(no_baits['angle_to_P'])*no_baits['dist_P']*np.exp(-(no_baits['dist_P']/lambd)**2)
baits_sort = no_baits.sort_values('dist_P')
N=300
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('gradient')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(400,1500)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(no_baits['dist_P'],no_baits['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av['dist_P'],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av['dist_P'],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av['dist_P'],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
# x= range(-xlim,xlim+1)
# legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
# line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

In [10]:
p_value

0.02223930581323196

In [31]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
zipped_list = zip(np.abs(baits['angle_to_P']),baits['straightness'])
sorted_zip = sorted(zipped_list)
densities_sort,curvatures_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
# ax.set_xlabel('curvilinear length ($\mu m$')
# ax.set_ylabel('straightness ($\mu m. \mu m^{-1}$)')
xlim = 180
# ax.set_xlim(400,1500)
# ax.set_ylim((0.97,1.01))
slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(0,xlim)

N=100
ax.scatter(densities_sort,curvatures_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(curvatures_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
from scipy.ndimage import uniform_filter1d
from scipy import stats
plt.close('all')
zipped_list = zip(baits['dist_P']*1.725/1000,baits['speed'])
sorted_zip = sorted(zipped_list)
densities_sort,curvatures_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('distance to P(mm)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 70
ax.set_ylim(0,400)
# ax.set_ylim((0.97,1.01))
slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(0,xlim)

N=300
ax.scatter(densities_sort,curvatures_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(curvatures_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
p_value

0.705453877735688